In [3]:
import numpy as np
from scipy.optimize import brentq

In [4]:
# Constants
gravConst = 4.302e-6  # M_sun^-1 (km/s)^2 kpc
R_sun = 8.1  # kpc
GeVcm3_to_Msunkpc3 = 2.685e7  # GeV/cm^3 to M_sun/kpc^3
Msunpc2_to_Msunkpc2 = 1.0e6  # M_sun/pc^2 to M_sun/kpc^2
H_0 = 72.1e-3  # (km/s)*kpc^-1
rho_crit = 3 * H_0**2 / (8 * np.pi * gravConst)  # in M_sun/kpc^3

In [5]:
def mass_nfw(rhoDM_sun,Rs,r):
    Rhos = (rhoDM_sun * GeVcm3_to_Msunkpc3) * (R_sun / Rs) * (1 + (R_sun / Rs))**2
    gr = np.log(np.maximum((Rs + r) / Rs, 1e-10)) - (r / (Rs + r + 1e-10))
    mass_dm = 4.0 * np.pi * Rhos * gr * Rs**3
    return mass_dm  # units of M_sun

In [6]:
def mass_single_exp(sigma_sun,Rb,r):
    Sigmab = (sigma_sun * Msunpc2_to_Msunkpc2) * np.exp(R_sun / Rb)
    M_vis = 2 * np.pi * Sigmab * Rb**2 * (1 - (1 + r / Rb) * np.exp(-r / Rb))
    return M_vis

In [7]:
def mass_total(rhoDM_sun,Rs,sigma_sun,Rb,r):
    M_dm = mass_nfw(rhoDM_sun,Rs,r)
    M_vis = mass_single_exp(sigma_sun,Rb,r)
    M_tot = M_dm + M_vis
    return M_tot  # units of M_sun

In [8]:
def density_at_r(rhoDM_sun,Rs,sigma_sun,Rb,r):
    return mass_total(rhoDM_sun,Rs,sigma_sun,Rb,r) / ((4 / 3) * np.pi * r**3)

In [9]:
def R_200(rhoDM_sun, Rs, sigma_sun, Rb):
    def equation(r):
        return density_at_r(rhoDM_sun, Rs, sigma_sun, Rb,r) - 200 * rho_crit
    
    # Need a numerical solver to find the root of the objective function
    r_low = 1e-3  # Lower bound for radius
    r_high = 500.0 # Upper bound for radius (adjust based on expected galaxy size)

    try:
        R_200_value = brentq(equation, r_low, r_high)
        return R_200_value
    except ValueError:
        print("Error: Could not find R_200 within the given bounds.")
        return None

In [10]:
def R_100(rhoDM_sun, Rs, sigma_sun, Rb):
    def equation(r):
        return density_at_r(rhoDM_sun, Rs, sigma_sun, Rb, r) - 100 * rho_crit
    
    # Need a numerical solver to find the root of the objective function
    r_low = 1e-3  # Lower bound for radius
    r_high = 500.0 # Upper bound for radius (adjust based on expected galaxy size)

    try:
        R_200_value = brentq(equation, r_low, r_high)
        return R_200_value
    except ValueError:
        print("Error: Could not find R_200 within the given bounds.")
        return None

In [ ]:
best_fit_params = [0.4, 20.0, 0.5, 1.0]  # Example parameters: rhoDM_sun, Rs, sigma_sun, Rb
rhoDM_sun, Rs, sigma_sun, Rb = best_fit_params
R200 = R_200(rhoDM_sun, Rs, sigma_sun, Rb)
M_200 = mass_total(rhoDM_sun, Rs, sigma_sun, Rb,R200)
R100 = R_100(rhoDM_sun, Rs, sigma_sun, Rb)
M_100 = mass_total(rhoDM_sun, Rs, sigma_sun, Rb,R100)

M_rsun_total = mass_total(rhoDM_sun, Rs, sigma_sun, Rb, R_sun)
M_rsun_dm = mass_nfw(rhoDM_sun, Rs,R_sun)
M_rsun_vis = mass_single_exp(sigma_sun, Rb, R_sun)

In [12]:
from IPython.display import display, Markdown

def to_latex_sci(value):
    """Converts a float to LaTeX scientific notation string."""
    mantissa, exponent = f"{value:.2e}".split('e')
    return f"{mantissa} \\times 10^{{{int(exponent)}}}"

# Format each variable
M200_latex      = to_latex_sci(M_200)
R200_latex      = f"{R200:.2f}"
M_rsun_total_latex = to_latex_sci(M_rsun_total)
M_rsun_dm_latex = to_latex_sci(M_rsun_dm)
M_rsun_vis_latex = to_latex_sci(M_rsun_vis)
M_100_latex     = to_latex_sci(M_100)
R100_latex      = f"{R100:.2f}"

# Display formatted output
display(Markdown(f"""
- $M_{{200}}^{{\\text{{total}}}} = {M200_latex}\\ M_\\odot$
- $R_{{200}}^{{\\text{{total}}}} = {R200_latex}\\ \\text{{kpc}}$
- $M_{{\\text{{total}}}}(r_\\odot) = {M_rsun_total_latex}\\ M_\\odot$
- $M_{{\\text{{DM}}}}(r_\\odot) = {M_rsun_dm_latex}\\ M_\\odot$
- $M_{{\\text{{visible}}}}(r_\\odot) = {M_rsun_vis_latex}\\ M_\\odot$
- $M_{{100}}^{{\\text{{total}}}} = {M_100_latex}\\ M_\\odot$
- $R_{{100}}^{{\\text{{total}}}} = {R100_latex}\\ \\text{{kpc}}$
"""))





- $M_{200}^{\text{total}} = 1.69 \times 10^{12}\ M_\odot$
- $R_{200}^{\text{total}} = 241.17\ \text{kpc}$
- $M_{\text{total}}(r_\odot) = 1.30 \times 10^{11}\ M_\odot$
- $M_{\text{DM}}(r_\odot) = 1.46 \times 10^{10}\ M_\odot$
- $M_{\text{visible}}(r_\odot) = 1.16 \times 10^{11}\ M_\odot$
- $M_{100}^{\text{total}} = 2.09 \times 10^{12}\ M_\odot$
- $R_{100}^{\text{total}} = 326.03\ \text{kpc}$


In [11]:
# import h5py

# filename = r'C:\Amogh\Research\TIFR Project\Mass Modelling\NFW_single_exp_analysis\Analysis_for_LPV\LPV_analysis_visible_gaussian_67_wideprior_dm\mcmc_chains.h5'

# with h5py.File(filename, 'r') as f:
#     print("HDF5 File Structure:\n")
#     def print_structure(name, obj):
#         print(name)
#     f.visititems(print_structure)


In [1]:
#code to obtain min chi2 from mcmc chains
import h5py
import numpy as np

filename = r'C:\Amogh\Research\TIFR Project\Mass Modelling\NFW_single_exp_analysis\Analysis_for_Eilers\Eilers_analysis_visible_gaussian_48_noprior_dm\mcmc_chains.h5'

with h5py.File(filename, 'r') as f:
    log_prob = f['log_prob'][()]
    chains = f['chains'][()]

    # Mask for valid log_probs
    valid_mask = np.isfinite(log_prob)
    valid_log_prob = log_prob[valid_mask]
    valid_chains = chains[valid_mask]  # 🛠️ align chains with valid log_probs

    # Convert to chi-squared
    chi2 = -2 * valid_log_prob
    min_index = np.argmin(chi2)
    min_chi2 = chi2[min_index]

    best_params = valid_chains[min_index]  # now this matches correctly

    print(f"Minimum chi-squared: {min_chi2:.4f}")
    print("Best-fit parameter values:")
    for i, val in enumerate(best_params):
        print(f"  Param {i + 1}: {val:.6f}")



Minimum chi-squared: 100.6882
Best-fit parameter values:
  Param 1: 0.475693
  Param 2: 5.799256
  Param 3: 22.341291
  Param 4: 2.302814
